In [1]:
import numpy as np
import matplotlib
from copy import deepcopy

## Chromosome Class

In [2]:
class Chromosome:
    
    def __init__(self, n,genes=None):
        self.size = n
        #this can generate invalid solutions
        if genes is None:
            self.genes = np.random.randint(low=0,high=n,size=n) 
        else:
            self.genes = genes
        #self.genes = np.arange(start=1,stop=9)
        #np.random.shuffle(self.genes)
        
        self.fitness = -1
        
        
    def __repr__(self):
        return "chromosome : {} fitness: {}\n".format(self.genes, self.fitness)
    
    def __len__(self):
        return self.size
    
        

## Selection

In [3]:
def tournament_selection(population, tournament_size=3):
        
        collection = [np.random.choice(population) for _ in range(tournament_size)]
        cost_function(collection)
        return (min(collection, key=compare_chromosome))
    



def compare_chromosome(chromosome):
    return chromosome.fitness

## Crossover

In [5]:

def crossover(population, crossover_method, selection_method, px = 0.9):
    
    offsprings = []
    new_population = []
    
    
    while len(offsprings) < 8:
        p1 = selection_method(population)
        p2 = selection_method(population)
        
        if np.random.rand() <= px :
            o1,o2 = crossover_method(p1.genes, p2.genes)
            
            offsprings.append(o1)
            offsprings.append(o2)
                        

    #print(offsprings)
    return offsprings


def PMX_crossover(parent_1, parent_2):
    size = len(parent_1)
    p1, p2 = [-1]*size, [-1]*size
    
    offspring_1 = deepcopy(parent_1)
    offspring_2 = deepcopy(parent_2)

    # Initialize the position of each indices in the individuals
    for i in range(size):
        p1[parent_1[i]] = i
        p2[parent_2[i]] = i
        
    # Choose crossover points
    from_index = np.random.randint(0, size-1)
    to_index = np.random.randint(from_index+1, size)
    
    
    # Apply crossover between cx points
    for i in range(from_index, to_index):
        # Keep track of the selected values
        temp1 = parent_1[i]
        temp2 = parent_2[i]
        # Swap the matched value
        offspring_1[i], offspring_1[p1[temp2]] = temp2, temp1
        offspring_2[i], offspring_2[p2[temp1]] = temp1, temp2
        # Position bookkeeping
        p1[temp1], p1[temp2] = p1[temp2], p1[temp1]
        p2[temp1], p2[temp2] = p2[temp2], p2[temp1]

    return Chromosome(size,genes=offspring_1), Chromosome(size,genes=offspring_2)



## Mutation

In [6]:
def mutate(population,pm=0.1):
    
    for chromosome in population:
        if np.random.rand() < pm:
            
            for i in chromosome.genes:
                if np.random.rand() < pm:
                    
                    if i >= 0 and i < len(chromosome)-1:
                        j = i+1
                    else:
                        j = i-1
                        
                    tmp = chromosome.genes[i]
                    chromosome.genes[i] = chromosome.genes[j]
                    chromosome.genes[j] = tmp
                        
        

## Select next generation

In [7]:
def select_next_population(population,offsprings):
    best1 = min(population, key=compare_chromosome)
    population.remove(best1)
    
    best2 = min(population, key=compare_chromosome)
    population.remove(best2)
    
    next_poulation = deepcopy(offsprings)
    next_poulation.append(best1)
    next_poulation.append(best2)
    
    return next_poulation

# Cost function (fitness function)

In [8]:
def cost_function(population): #fitness function -> count the number of threats for each chromosome
    # calculate row and column clashes
    # just subtract the unique length of array from total length of array
    # [1,1,1,2,2,2] - [1,2] => 4 clashes
    
    costs = []
    
    for chromosome in population:
        threats = 0;
        row_threats = abs(len(chromosome) - len(np.unique(chromosome.genes))) #threats count in row and column
        threats += row_threats

        for i in range(chromosome.size): # count number of threats in diagonal
            for j in range(chromosome.size):
                if ( i == j): # don't want to fight with his own
                    continue
                    
                x = abs(i-j)
                y = abs(chromosome.genes[i] - chromosome.genes[j])
                if(x == y):
                    #print('{},{} -- x: {}, y:{}'.format(i,j,x,y))
                    threats += 1
        chromosome.fitness = threats

    return costs

## Genetic algorithm

In [16]:
def begin_genetic_algorithm(queen_size=8,
                            population_size=10, crossover_probablity=0.9,mutation_probablity=0.1):
    
    population = [Chromosome(queen_size) for _ in range(population_size)]
    
    population.append(Chromosome(queen_size))
    population[-2].genes = np.array([1,4,6,3,0,4,5,5])
    #population[-1].genes = np.array([1,4,6,3,0,7,5,2])
    
    #print(np.setdiff1d(population[-1].genes, population[-2].genes))
    #print(population[1].genes.tolist().index(5))
    current_generation = 0 
    
    # Compute cost function of all indeviduals
    cost_function(population)
    
    best = min(population, key=compare_chromosome) 
    
    #print(population)
    #print(tournament_selection(population=population))
    #print(tournament_selection(population=population))
    #print(tournament_selection(population=population))
    #print(tournament_selection(population=population))
    
    
    #print(population)
    
    while best.fitness != 0:
        
        # Generate offsprings
        new_pop = crossover(population=population, crossover_method=PMX_crossover 
                            ,selection_method=tournament_selection,px=crossover_probablity)
        
        # Mutation
        mutate(population,pm=mutation_probablity)
        
        # Select next generation
        population = select_next_population(new_pop,population)
        current_generation += 1
        
        cost_function(population)
        
        best = min(population, key=compare_chromosome)
        
        print("Generation: {} , Best fitness: {}".format(current_generation,best.fitness))
    
    print("Generation: {} , Best fitness: {}".format(current_generation,best))


    %%time
    

## Just run this for genetic algorithm

In [17]:
begin_genetic_algorithm(queen_size=8,population_size=10, crossover_probablity=0.9,mutation_probablity=0.1)

Generation: 1 , Best fitness: 4
Generation: 2 , Best fitness: 4
Generation: 3 , Best fitness: 4
Generation: 4 , Best fitness: 4
Generation: 5 , Best fitness: 4
Generation: 6 , Best fitness: 4
Generation: 7 , Best fitness: 4
Generation: 8 , Best fitness: 4
Generation: 9 , Best fitness: 4
Generation: 10 , Best fitness: 4
Generation: 11 , Best fitness: 4
Generation: 12 , Best fitness: 4
Generation: 13 , Best fitness: 4
Generation: 14 , Best fitness: 4
Generation: 15 , Best fitness: 4
Generation: 16 , Best fitness: 4
Generation: 17 , Best fitness: 4
Generation: 18 , Best fitness: 4
Generation: 19 , Best fitness: 4
Generation: 20 , Best fitness: 4
Generation: 21 , Best fitness: 4
Generation: 22 , Best fitness: 4
Generation: 23 , Best fitness: 4
Generation: 24 , Best fitness: 4
Generation: 25 , Best fitness: 4
Generation: 26 , Best fitness: 4
Generation: 27 , Best fitness: 4
Generation: 28 , Best fitness: 4
Generation: 29 , Best fitness: 4
Generation: 30 , Best fitness: 4
Generation: 31 , Be

## BackTracking

In [57]:
def solveQU(board, column, queen_size):   
    if column >= queen_size: 
        return True   
    for i in range(queen_size): 
        if isSafe(board, i, column, queen_size): 
            board[i][column] = 1 
            if solveQU(board, column+1, queen_size) == True: 
                return True 
            board[i][column] = 0 
    return False 

def isSafe(board, row, column, queen_size):  
    for i in range(column): 
        if board[row][i] == 1: 
            return False 
        
    for i,j in zip(range(row,-1,-1), range(column,-1,-1)): 
        if board[i][j] == 1: 
            return False 
    for i,j in zip(range(row,queen_size,1), range(row,-1,-1)): 
        if board[i][j] == 1: 
            return False 
    return True 

def start(queen_size): 
    
    board = board = np.zeros((queen_size,queen_size),dtype=int)
    
    if solveQU(board, 0, queen_size): 
        print(board)
        return
    else:
        print ("There is no solution ") 
      


start(8)

[[1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1]
 [0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0 0]]


## Analytical

In [1]:
import numpy as np

n = 8

board = np.zeros(shape=(n, n))

def putQueen(y, x) :
    board[y - 1, x - 1] = 1

def handleEvenBoard(board) :
    if n % 2 == 0 and n % 6 != 2 :
        for i in range(1, n // 2 + 1) :
            putQueen(2 * i, i)
            putQueen(2 * i - 1, n // 2 + i)
    elif n % 2 == 0 and n % 6 != 0 :
        for i in range(1, n // 2 + 1) :
            print(i)
            putQueen(1 + (2 * i + n // 2 - 3) % n, i)
            putQueen(n - (2 * i + n // 2 - 3) % n, n + 1 - i)

if n % 2 == 0 :
    handleEvenBoard(board)
elif n % 2 == 1 :
    putQueen(n, n)
    n = n - 1
    handleEvenBoard(board)

print(board)

1
2
3
4
[[0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]]
